In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import imread
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow_hub as hub


In [2]:
normal_list = os.listdir("/content/drive/My Drive/ChestXRays/NORMAL")
diseased_list = os.listdir("/content/drive/My Drive/ChestXRays/PNEUMONIA")
nzeros = [0]*len(normal_list)
dones = [1]*len(diseased_list)

xray_images = normal_list + diseased_list
xrays = np.array(xray_images)
classification_values = nzeros+dones
target = np.array(classification_values)
len(xrays), len(target)


(5216, 5216)

In [3]:
tablet = pd.DataFrame({"Xrays":xrays, "target":target})
tablet.head()

,Xrays,target
0,NORMAL2-IM-0815-0001.jpeg,0
1,NORMAL2-IM-0819-0001.jpeg,0
2,NORMAL2-IM-0824-0001.jpeg,0
3,NORMAL2-IM-0832-0001.jpeg,0
4,NORMAL2-IM-0822-0001.jpeg,0


In [4]:
table = tablet.sample(frac=1).reset_index(drop=True)
table.head()

,Xrays,target
0,person939_bacteria_2864.jpeg,1
1,person155_virus_307.jpeg,1
2,IM-0559-0001.jpeg,0
3,person1493_bacteria_3899.jpeg,1
4,IM-0551-0001-0001.jpeg,0


In [5]:
table = pd.read_csv("/content/drive/My Drive/ChestXRays/table.csv")
table.head()

,Xrays,target
0,person1224_virus_2074.jpeg,1
1,person354_bacteria_1634.jpeg,1
2,person472_bacteria_2008.jpeg,1
3,IM-0691-0001.jpeg,0
4,NORMAL2-IM-0589-0001.jpeg,0


In [6]:
filenames = ["/content/drive/My Drive/ChestXRays/XRays/" + fname for fname in table["Xrays"]]
len(filenames)
labels = table["target"].to_numpy()
binary_labels = [label == 1 for label in labels]
labels[0], binary_labels[0]

(1, True)

In [7]:
np.unique(labels)

array([0, 1])

In [8]:
X= filenames
y= binary_labels

In [39]:
NUM_IMAGES = 1000 #@param {type:"slider", min:1000, max:5000, step:500}

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X[:NUM_IMAGES],y[:NUM_IMAGES], test_size = 0.2, random_state=42)


In [41]:
IMG_SIZE = 224
def tensorify(image_path):
  """
  turn an image into a tensor
  """
  image = tf.io.read_file(image_path)
  image = tf.image.decode_jpeg(image, channels=3)
  image = tf.image.convert_image_dtype(image, tf.float32)
  image = tf.image.resize(image, size=[IMG_SIZE, IMG_SIZE])
  return image


def get_label(image_path, label):
  """
  gives label
  """
  image = tensorify(image_path)
  return image, label



In [42]:

BATCH_SIZE = 32

# Create a function to turn data into batches
def create_batches(X, y=None, batch_size=BATCH_SIZE,  test_data=False):
  """
  Creates batches of data out of image (X) and label (y) pairs.
  Shuffles the data if it's training data but doesn't shuffle if it's validation data.
  Also accepts test data as input (no labels).
  """
  if test_data:
    print("Creating test data batches...")
    data = tf.data.Dataset.from_tensor_slices((tf.constant(X), # filepaths
                                               tf.constant(y))) # labels
    data_batch = data.map(get_label).batch(BATCH_SIZE)
    return data_batch
  else:
    print("Creating training data batches...")
    data = tf.data.Dataset.from_tensor_slices((tf.constant(X),
                                               tf.constant(y)))
    data = data.shuffle(buffer_size=len(X))
    data = data.map(get_label)
    data_batch = data.batch(BATCH_SIZE)
  return data_batch
  

In [43]:
train_data = create_batches(X_train, y_train)
test_data = create_batches(X_test, y_test)

Creating training data batches...
Creating training data batches...


In [44]:
train_data.element_spec

(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None),
 TensorSpec(shape=(None,), dtype=tf.bool, name=None))

In [45]:
INPUT_SHAPE = [None, 224, 224, 3]
OUTPUT_SHAPE = 1
MODEL_URL = "https://tfhub.dev/google/imagenet/mobilenet_v2_130_224/classification/4"

In [46]:
def create_model(input_shape=INPUT_SHAPE, output_shape=OUTPUT_SHAPE, model_url=MODEL_URL):
  print("Building model with:", MODEL_URL)

  model = tf.keras.Sequential([
    hub.KerasLayer(MODEL_URL), # Layer 1 (input layer)
    tf.keras.layers.Dense(units=OUTPUT_SHAPE,
                          activation="sigmoid") # Layer 2 (output layer)
  ])

  # Compile the model
  model.compile(
      loss=tf.keras.losses.BinaryCrossentropy(),
      optimizer=tf.keras.optimizers.Adam(),
      metrics=[tf.keras.metrics.AUC(
        name='auc_precision_recall', curve='PR', num_thresholds=10000)]
  )

  # Build the model
  model.build(INPUT_SHAPE)

  return model

In [47]:
model = create_model()
model.summary()

Building model with: https://tfhub.dev/google/imagenet/mobilenet_v2_130_224/classification/4
Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_9 (KerasLayer)   multiple                  5432713   
_________________________________________________________________
dense_9 (Dense)              multiple                  1002      
Total params: 5,433,715
Trainable params: 1,002
Non-trainable params: 5,432,713
_________________________________________________________________


In [48]:
NUM_EPOCHS = 100 #@param {type:"slider", min:10, max:100, step:10}

In [49]:
%load_ext tensorboard
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="auc_precision_recall",
                                                  patience=3)
def train_model():
  """
  Trains a given model and returns the trained version.
  """
  model = create_model()
  model.fit(x=train_data,
            epochs=NUM_EPOCHS,
            validation_data=test_data,
            validation_freq=1,
            callbacks=[early_stopping])
  return model

model = train_model()

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
Building model with: https://tfhub.dev/google/imagenet/mobilenet_v2_130_224/classification/4
Epoch 1/100
25/25 [==============================] - 17s 677ms/step - loss: 0.4384 - auc_precision_recall: 0.9398 - val_loss: 0.1959 - val_auc_precision_recall: 0.9922
Epoch 2/100
25/25 [==============================] - 16s 634ms/step - loss: 0.1798 - auc_precision_recall: 0.9920 - val_loss: 0.1536 - val_auc_precision_recall: 0.9953
Epoch 3/100
25/25 [==============================] - 16s 638ms/step - loss: 0.1467 - auc_precision_recall: 0.9947 - val_loss: 0.1429 - val_auc_precision_recall: 0.9953
Epoch 4/100
25/25 [==============================] - 16s 642ms/step - loss: 0.1230 - auc_precision_recall: 0.9965 - val_loss: 0.1475 - val_auc_precision_recall: 0.9957


In [58]:

# Create a function to save a model
def save_model(model, suffix=None):
  """
  Saves a given model in a models directory and appends a suffix (string).
  """

  model_path = "/content/drive/My Drive/ChestXRays" + "/" + suffix + ".h5" 
  print(f"Saving model to: {model_path}...")
  model.save(model_path)
  return model_path

save_model(model, suffix="PneumoniaChestXRayClassifier")

Saving model to: /content/drive/My Drive/ChestXRays/PneumoniaChestXRayClassifier.h5...


'/content/drive/My Drive/ChestXRays/PneumoniaChestXRayClassifier.h5'